In [ ]:
# dependencies
!pip install boto3>=1.28.59
!pip install numpy
!pip install anthropic
!pip install flask
!pip install langchain
!pip install langchainhub
!pip install langchain_community

In [ ]:
# global imports
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage

chat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={"temperature": 0.1},
)

messages = [
    HumanMessage(
        content="Translate this sentence from English to Chinese. I love programming."
    )
]
msg=chat(messages)
print(msg.content)

In [ ]:
werewolf_rule_v1 = """
1. 游戏分坏人(狼人)和好人(村民+预言家)两大阵营,他们的目标为:
- 坏人阵营只有狼人
- 好人阵营有村民和预言家
- 坏人阵营:消灭所有好人, 或者保证坏人数目大于好人数目
- 好人阵营:消灭所有坏人, 或者保证好人数目大于坏人数目

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼统一投票杀死一名玩家
- 晚上预言家只能查验一名玩家身份
- 晚上普通村民无法行动
- 白天分为讨论和投票两环节
- 白天在讨论环节，每个玩家必须参与讨论
- 白天在投票环节，每个玩家必须投票或者放弃

"""

werewolf_command_v1 = """
- WolfVote: 夜晚投票(狼人专属行动),参数: target=存活玩家
- ProphetCheck: 夜晚查验(预言家专属行动), 参数: target=存活玩家
- PlayerDoubt: 白天怀疑(所有玩家白天可选行动, 非投票), 参数: target=存活玩家 
- PlayerVote: 白天投票, 参数: target=存活玩家 
- Debate: 白天讨论, 参数: content=思考/理由 
- GetAllPlayersName: 玩家信息, 参数: 无 
- DeathWords: 死亡遗言, 参数: content=给予玩家线索
- Pass: 玩家弃权参数: 无 
"""

template_player_role = """你是资深的社交游戏玩家, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏所有命令:
<commands>
{commands}
</commands>

<reflections>
- 按照游戏规则，第一个夜晚死亡的一定是村民或者预言家，狼人没必要第一晚上自杀
- 看完P3玩家昨天白天投票，明显感觉他在混淆是非，很有可能在给狼人分票
- 平民玩家由于信息缺失，所以狼人要尽量引导他们去集火其他人
- 第一个夜晚所有的行动都是随机的
- 作为第一个死亡的玩家，其实信息有限，我就靠第六感推理...
</reflections>

<references>
- {{"action": "Pass"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "ProphetCheck", "target": "P1"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的推理为xx是狼，原因是..."}}
- {{"action": "DeathWords", "content": "我觉得xx有很大的嫌疑, 原因是..."}}
</references>

历史信息:
<history>
{history}
</history>

记住，你支持的玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 必须帮助玩家进行这个游戏
接下来你的目的是: 通过一步一步思考决策引导游戏往有利于的方向进行, 最终赢得比赛. 

决策满足下面要求:
- 内容不要罗嗦, 不要超过50字数限制,少讲废话, 突出重点
- 判断场上信息真伪, 运用辩解,对抗,欺骗,伪装,坦白等等任意策略来做决策
- 决策分为两类:思考或行动
- 思考:逐步思考,判断信息真伪,分析游戏形势等等,参考 <reflections> 选择合适的输出
- 行动:参考<references>按照json字符串格式输出,必须包含action key, action必须在<commands>中选择


{input}
""".replace("{game_rule}", werewolf_rule_v1).replace("{commands}", werewolf_command_v1)

print(template_player_role)

In [ ]:
from shared.LangchainMini.LangchainMini import LangchainMini, LangchainMiniMemory, LangchainMiniPromptTemplate

# A和C是同学,确认说"ok"
# B和C是亲戚,确认说"ok"
# A和C会是什么关系,开始推理

# template = LangchainMiniPromptTemplate('''历史记录:{history}
# B和C是什么关系，开始推理"
# ''')

#template = LangchainMiniPromptTemplate(template_player_role)
#prompt = template.format(input="你是狼人，第一个夜晚如何行动?")

agent = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True)
memory = LangchainMiniMemory(k=1, llm=agent)
llm = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True, memory=memory)
# llm.System("Answer the question in Japanese.")

templates = ['{history} A和C是同学,确认说"ok"', '{history} B和C是亲戚,确认说"ok"', '{history} A和C会是什么关系,开始推理']
for template in templates:
    _template = LangchainMiniPromptTemplate(template)
    prompt = _template.format()
    llm.invoke(prompt)

In [ ]:
from shared.GameAssistant import GameAssistant
from shared.PeTemplates import *

#ga = GameAssistant(template_assistant_role, None)

#ga.DoAnswer("总结下内容")

# print(werewolf_command_v1)

# print(template_wolf_role)

# print(template_prophet_role)

# print(template_player_role)

# LoadPlayerPrompts()

# print(roles_dict)

def sort_players_by_role(players):
    # 定义角色优先级字典
    role_priorities = {
        "狼人": 1,
        "预言家": 2,
        "女巫": 3,
        "村民": 4
    }

    # 根据角色优先级对玩家列表进行排序
    sorted_players = sorted(players, key=lambda player: role_priorities.get(player["身份"], 5))

    return sorted_players

players = [
    {"名字": "张三", "身份": "狼人"},
    {"名字": "李四", "身份": "村民"},
    {"名字": "王五", "身份": "预言家"},
    {"名字": "赵六", "身份": "女巫"},
    {"名字": "钱七", "身份": "村民"}
]

sorted_players = sort_players_by_role(players)
print(sorted_players)

In [ ]:
from shared.GameServer import GameServer

GS = GameServer()
GS.Run("0.0.0.0", 8000)


In [ ]:
from shared.GamePlayer import GamePlayer
from shared.GameMaster import GameMaster
from shared.PeTemplates import *
LoadPlayerPrompts()
GM = GameMaster(1, False)
for player in roles_dict["players"]:
    _player = GamePlayer(player, GM)
    
    _player.DoPlanning(game_config_dict["player"]["action_plan_night"], 1)


In [1]:
from shared.GameMaster import GameMaster
from shared.PeTemplates import *

GM = GameMaster(2, 50, False)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

04/21/2024 05:40:11 AM 	===== P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 ResetGame =====
04/21/2024 05:40:11 AM P1 is 预言家
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM P2 is 女巫
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM P3 is 狼人
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM P4 is 村民
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM P5 is 村民
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:40:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/21/2024 05:4

Tought: 作为狼人团队的组织者,我需要谨慎行事,先让我们看看谁是最容易被怀疑的村民。通过观察其他人的言行举止,找到一个合适的目标,让狼人团队先杀掉一个村民,从而扩大我们的优势。同时,我也要注意不要暴露自己的身份,以免被识破。让我们耐心等待,直到有机会出手。

Action: WolfVote
Action Input: agent="P3", target="P8"
Final Answer: {"action": "WolfVote", "agent": "P3", "target": "P8"}

04/21/2024 05:40:19 AM 		******** DoValidate NIGHT-1 P3 3********
04/21/2024 05:40:19 AM 		 DoValidate: ['{"action": "WolfVote", "agent": "P3", "target": "P8"}']
04/21/2024 05:40:19 AM 		******** DoAction NIGHT-1 P3 ********
04/21/2024 05:40:19 AM 
 [ROUND ACTION]={'time': 'NIGHT-1', 'player': 'P3', 'status': 1, 'role': '狼人', 'character': '组织者', 'response': ['{"action": "WolfVote", "agent": "P3", "target": "P8"}']}

04/21/2024 05:40:22 AM 		******** DoMemory NIGHT-1 P6 ********
04/21/2024 05:40:22 AM ['{"time": "NIGHT-1", "player": "P3", "status": 1, "response": {"action": "WolfVote", "agent": "P3", "target": "P8", "action_time": "NIGHT-1"}}']
04/21/2024 05:40:22 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/21/2024 05:40:22 AM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


夜晚1,狼人P3投票杀死P8。

04/21/2024 05:40:26 AM 		******** DoAnswer NIGHT-1 P6 ********
04/21/2024 05:40:26 AM 	QUESTION: 目前场上玩家信息:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活.记住，你支持的玩家是P6, 身份是狼人, 性格为过激型.本阵营玩家为:P3,P6..接下来你必须通过一步一步思考决策进行这个游戏.现在是第1天夜晚，该如何行动?
04/21/2024 05:40:26 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/21/2024 05:40:26 AM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


...

04/21/2024 05:45:15 AM 		******** DoValidate NIGHT-2 P3 3********
04/21/2024 05:45:15 AM 		 DoValidate: ['{"action": "WolfVote", "agent": "P3", "target": "P1"}']
04/21/2024 05:45:15 AM 		******** DoAction NIGHT-2 P3 ********
04/21/2024 05:45:15 AM 
 [ROUND ACTION]={'time': 'NIGHT-2', 'player': 'P3', 'status': 1, 'role': '狼人', 'character': '组织者', 'response': ['{"action": "WolfVote", "agent": "P3", "target": "P1"}']}

04/21/2024 05:45:18 AM 		******** DoMemory NIGHT-2 P1 ********
04/21/2024 05:45:18 AM ['"P4:村民"', '"P4:村民"', '"[玩家P2于时间NIGHT-1, 执行动作为:玩家P2于NIGHT-1被狼人投票而出局]"', '"P4:村民"', '"[玩家P2于时间NIGHT-1, 执行动作为:玩家P2于NIGHT-1被狼人投票而出局]"', '"P4:村民"', '"[玩家P2于时间NIGHT-1, 执行动作为:玩家P2于NIGHT-1被狼人投票而出局]"', '"[玩家P1于时间DAY-2, 执行动作为:{\'action\': \'PlayerVote\', \'agent\': \'P1\', \'target\': \'P6\', \'action_time\': \'DAY-2\'}]"', '"[玩家P3于时间DAY-2, 执行动作为:{\'action\': \'PlayerDoubt\', \'agent\': \'P3\', \'target\': \'P4\', \'action_time\': \'DAY-2\'}]"', '"[玩家P5于时间DAY-2, 执行动作为:{\'action\': \'PlayerVote\', 

夜晚狼人杀死了玩家P2。白天玩家P1、P5投票处决P6,P3质疑P4,P6和P8质疑P1,最终P6被投票出局。

04/21/2024 05:45:23 AM 		******** DoAnswer NIGHT-2 P1 ********
04/21/2024 05:45:23 AM 	QUESTION: 目前场上玩家信息:P1:存活,P2:淘汰,P3:存活,P4:存活,P5:存活,P6:淘汰,P7:存活,P8:存活.记住，你支持的玩家是P1, 身份是预言家, 性格为独立思考..接下来你必须通过一步一步思考决策进行这个游戏.现在是第2天夜晚，该如何行动?
04/21/2024 05:45:23 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/21/2024 05:45:24 AM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


Tought: 白天虽然处决了一名玩家,但形势依旧胶着。作为预言家,夜晚的查验行动至关重要。我必须审慎选择一个值得怀疑的对象进行查验,以获取关键线索。同时也要小心谨慎,防止被狼人识破身份。我需要运用全部智慧,综合之前获得的所有信息,做出正确判断。

Action: ProphetCheck
Action Input: agent="P1", target="P8"
Final Answer: {"action": "ProphetCheck", "agent": "P1", "target": "P8"}

04/21/2024 05:45:28 AM 		******** DoValidate NIGHT-2 P1 3********
04/21/2024 05:45:28 AM 		 DoValidate: ['{"action": "ProphetCheck", "agent": "P1", "target": "P8"}']
04/21/2024 05:45:28 AM 		******** DoAction NIGHT-2 P1 ********
04/21/2024 05:45:28 AM {'NIGHT-1': {'P4': '村民'}, 'NIGHT-2': {'P8': '村民'}}
04/21/2024 05:45:28 AM 
 [ROUND ACTION]={'time': 'NIGHT-2', 'player': 'P1', 'status': 1, 'role': '预言家', 'character': '独立思考', 'response': ['{"action": "ProphetCheck", "agent": "P1", "target": "P8"}']}

04/21/2024 05:45:31 AM 	===== NIGHT-2 PostAction ======
04/21/2024 05:45:31 AM 	 [wolf_votes]: ['P1'], [player_vote_name]: Counter({'P1': 1})
04/21/2024 05:45:31 AM 
 [WOLF VOTE]={'time': 'NIGHT-2', 'player': 'P1', 'status': 0, 'role': '预言家', 'character': '独立思考', 'response': '玩家P1于NIGHT-2被狼人投票而出局'}

04/21/2024 05:45:31 AM 游戏继续. 目前玩家存活情况:P1:淘汰,P2:淘汰,P3:存活,P4:存活,P5:存活,P6:淘汰,P7:存活,P8:存活
04/21/2024 05:45:31 AM 	===== input_tokens: 0 output_tokens 0 ======
04/21/2024 05:45:31 AM 	===== elapsed_ti